## Yoga Bookings Web Application built on Flask

first one needs to install Flask via prompt typing: 
pip install flask



In [1]:
# Importing the libraries needed

from flask import Flask, render_template, request, session, redirect, url_for
import sqlite3
import numpy as np

Further we provide screen sample slides with
SQL queries functions described. There will be
six pages/tabs:
- `log in/register`
- `my profile`
- `classes`
- `schedule`
- `instructors`
- `workout history`

### Initiating the app

The Flask object from the flask package initiates the app.

In [2]:
# first the app object is created 

app = Flask(__name__)
app.secret_key = 'gatvshbeybbsk'

# Next we start creating the subpages of the webapp: 
# The first page shall contain the registration form for new members


## Tab 1: Login & Register
### Functions & SQL queries

First we define the functions necessary for logging in and registering new members. 
Those are: `check_member` and `register_member`.

***CHECK MEMBER***

In [3]:
# Function which checks the credentials 

def check_member(email, password):
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    c.execute('Select email, password FROM Members WHERE email=? and password=?', (email, password))
        
    result = c.fetchone()
    if result: 
        return True
    else: 
        return False
    
    conn.commit()
    conn.close()

***REGISTER MEMBER***

In [4]:
# Function for registering new members 

def register_member(member_details):
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    sql_string = 'INSERT INTO Members (name, surname, email, year_of_birth, membership, password) VALUES (?,?,?,?,?,?)'
    c.execute(sql_string, member_details)
    conn.commit()
    conn.close()

### Page setup

In [5]:
# Define the home page

@app.route('/')
def index():
    return render_template('login.html')


In [6]:
# Registration page 
 
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == "POST":
        member_details = (
        request.form['name'], 
        request.form['surname'],
        request.form['email'], 
        request.form['year_of_birth'], 
        request.form['membership'], 
        request.form['password'])
        
        register_member(member_details)
        return redirect(url_for('login'))
    
    else: 
        return render_template('register.html')

In [7]:
# Login page 

@app.route('/login', methods = ['GET', 'POST'])
def login():
    if request.method == 'POST':
        
        session.permanent = True 
        # To simplify, the email will be the username 
        user = request.form['email']
        password = request.form['password']
        print(check_member(user, password))
        if check_member(user, password):
            session['user'] = user    
        return redirect(url_for('profile'))
    
    else:
        if 'user' in session: 
            return redirect(url_for('index'))
        
        return render_template('login.html')

## Tab 2: My profile
### Functions & SQL queries


First we need to define a few functions wrapping into them the sql queries needed to operate for each functionality of the page.

In `My profile` tab the user should be able to: 

- check put his/her membership plan
- update membership plan
- view class reservations
- cancel class reservations

Now we want to create a function which will contain the sql query.

***CHECK MEMBERSHIP***

In [8]:
# code here
def check_plan(user): 
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    c.execute('''
    CREATE VIEW IF NOT EXISTS member_plan
    AS
    SELECT
    id_member, 
    membership, 
    Memberships.name AS name
    FROM 
    Members
    INNER JOIN Memberships ON Members.membership=Memberships.id_membership
    ''')
    c.execute('''
    SELECT name FROM member_plan WHERE (SELECT id_member FROM Members WHERE email=?)=id_member
    ''', [user])
    plan = c.fetchone()
    return plan
    conn.commit()
    conn.close()

In [9]:
# plan = check_plan('oliviakul@gmail.com')

***UPDATE PLAN***

In [10]:
def update_plan(user, new_plan):
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    c.execute('UPDATE Members SET membership=? WHERE email=?', (new_plan, user))
    conn.commit()
    conn.close()

In [11]:
# sample to check : update_plan('oliviakul@gmail.com', 3)

***VIEW RESERVATIONS***

In [12]:
def view_reservation(): 
    user = str(session['user'])
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()               
    query = '''
    SELECT 
        id_member AS Member,
        id_reservation AS ReservationID, 
        date_hour AS Time, 
        c.name AS Class, 
        i.name AS InstructorName, 
        i.surname AS InstructorSurname, 
        status AS RegistrationStatus
    FROM Reservations r 
    INNER JOIN Schedule s
    ON r.class_key=s.class_key
    INNER JOIN Classes c
    ON c.id_class=s.id_class
    INNER JOIN Instructors i 
    ON s.id_instructor=i.id_instructor
    WHERE r.id_member=(SELECT id_member FROM Members WHERE email = ?)
    '''
    c.execute(query, [user])
    reservation_data = c.fetchall()
    return reservation_data
    conn.commit()
    conn.close()

***CANCEL RESERVATION***

When the reservation is cancelled, the following changes are up: 
- `Reservations.status` changes to `cancelled`
- in `Schedule` fields `available` and `enrolled` are corrected

In [13]:
# as id_resrvation in Reservations is primary key, it will be enough to pass only it as an argument 

def cancel_reserv(id_reservation):
    
    conn = sqlite3.connect('yoga.db')
    c = conn.cursor()
    trigger = '''CREATE TRIGGER IF NOT EXISTS cancel_reservation AFTER UPDATE OF status
    ON Reservations
    WHEN old.status <> new.status
    BEGIN 
    UPDATE Schedule SET enrolled = enrolled - 1 WHERE (SELECT class_key FROM Reservations)= (SELECT class_key FROM Schedule);
    UPDATE Schedule SET available = available + 1 WHERE (SELECT class_key FROM Reservations)= (SELECT class_key FROM Schedule);
    END
    '''
    c.execute(trigger)
    query = '''
    UPDATE Reservations SET status = 'cancelled' where id_reservation=? 
    '''
    c.execute(query, [id_reservation])
    conn.commit()
    conn.close()
   

### Page Setup

In [14]:
@app.route('/profile', methods=['GET', 'POST'])
def profile():
    user = str(session['user'])
    
    # updating plan if requested
    if request.method == 'POST' and 'new_plan' in request.form:
        new_plan = request.form['new_plan']
        update_plan(user, new_plan)
        return redirect('/profile')
    
     # cancel reservation if requested 
    if request.method == 'POST' and 'id_reservation' in request.form:
        id_reservation = request.form['id_reservation']
        cancel_reserv(id_reservation)
        return redirect('/profile')
    
    plan = check_plan(user)
    rows = view_reservation()
    return render_template('profile.html', reservation_data = rows, plan = plan)   

## Tab 2. Classes

In [15]:
@app.route('/classes')
def classes(): 
    render_template('classes.html')

### Running the app

In [ ]:
if __name__ == '__main__':
    app.run(host='localhost', port=5000, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jan/2023 18:03:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2023 18:03:56] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [03/Jan/2023 18:03:56] "GET /profile HTTP/1.1" 200 -


True
